In [18]:
import pandas as pd
import numpy as np
import re

In [19]:
file = "hg002"

In [20]:
vcfeval_goal = pd.read_csv(file+'/summary_vcfeval_'+file+'.csv', skiprows = 0, index_col = 0)
vcfeval_goal[0:8]

,Precision,Recall,F1_Score,TP_Eval,TP_Base,FP,FN,Stratifier,IGV_Session,UNK,Name,Truth_Set,Summary_Type,Comparison_Engine,IndelLength
Type,,,,,,,,,,,,,,,
SNP,0.990900,0.994000,0.992500,3234532,3234812,29632,19540,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,772546.0,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
INDEL,0.988400,0.989200,0.988800,462828,462623,5452,5036,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,458723.0,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HetIndel,0.985112,0.987351,0.986230,284319,284204,4297,3641,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HomVarIndel,0.994636,0.993016,0.993825,177457,177437,957,1248,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HetSNP,0.985316,0.994106,0.989691,1954746,1954911,29131,11591,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HomVarSNP,0.999576,0.993827,0.996693,1279800,1279844,543,7950,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
NaN,0.989058,0.992816,0.990933,225244,225130,2492,1629,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,deletion,VcfEval,NaN
NaN,0.993076,0.991650,0.992362,209977,209956,1464,1768,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,insertion,VcfEval,NaN


In [21]:
vcfdist_tsv = pd.read_csv(file+'/precision-recall-summary.tsv', skiprows = 0, index_col = 0)
#vcfdist_tsv

In [22]:
#https://www.geeksforgeeks.org/python-convert-tsv-to-csv-file/
tsv_file=file+'/precision-recall-summary.tsv'
 
# reading given tsv file
vcfdist_csv=pd.read_table(tsv_file,sep='\t')
 
vcfdist_csv

,VAR_TYPE,MIN_QUAL,TRUTH_TP,QUERY_TP,TRUTH_FN,QUERY_FP,PREC,Unnamed: 7,RECALL,Unnamed: 9,F1_SCORE,F1_QSCORE
0,SNP,0,4514760,4514758,32255,35458,0.992188,0.992873,0.992530,21.266926,NaN,NaN
1,SNP,56,4511137,4511135,35878,28490,0.993706,0.992075,0.992890,21.481232,NaN,NaN
2,INDEL,0,663711,663701,5110,3658,0.993758,0.991510,0.992633,21.326944,NaN,NaN
3,INDEL,32,663653,663643,5168,3570,0.993891,0.991418,0.992653,21.338766,NaN,NaN


In [23]:
vcfdist_csv=vcfdist_csv.rename(columns={"VAR_TYPE":"Type", "MIN_QUAL": "MinQual", "TRUTH_TP": "TruthTP", "QUERY_TP":"QueryTP", "TRUTH_FN":"TruthFN", "QUERY_FP":"QueryFP", "PREC":"Precision", "Unnamed: 7":"Recall", "RECALL":"F1Score", "Unnamed: 9":"F1QScore", "F1_SCORE":"unnamed1", "F1_QSCORE":"unnamed2"})

vcfdist_csv

,Type,MinQual,TruthTP,QueryTP,TruthFN,QueryFP,Precision,Recall,F1Score,F1QScore,unnamed1,unnamed2
0,SNP,0,4514760,4514758,32255,35458,0.992188,0.992873,0.992530,21.266926,NaN,NaN
1,SNP,56,4511137,4511135,35878,28490,0.993706,0.992075,0.992890,21.481232,NaN,NaN
2,INDEL,0,663711,663701,5110,3658,0.993758,0.991510,0.992633,21.326944,NaN,NaN
3,INDEL,32,663653,663643,5168,3570,0.993891,0.991418,0.992653,21.338766,NaN,NaN


In [24]:
#Type Precision Recall F1_Score TP_Eval TP_Base FP FN Stratifier IGV_Session UNK Name Truth_Set Summary_Type Comparison_Engine IndelLength
vcfdist_reorg = vcfdist_csv[["Type","MinQual","Precision", "Recall", "F1Score", "QueryTP", "TruthTP", "QueryFP", "TruthFN"]]
vcfdist_reorg

,Type,MinQual,Precision,Recall,F1Score,QueryTP,TruthTP,QueryFP,TruthFN
0,SNP,0,0.992188,0.992873,0.992530,4514758,4514760,35458,32255
1,SNP,56,0.993706,0.992075,0.992890,4511135,4511137,28490,35878
2,INDEL,0,0.993758,0.991510,0.992633,663701,663711,3658,5110
3,INDEL,32,0.993891,0.991418,0.992653,663643,663653,3570,5168


In [25]:
vcfdist_final=vcfdist_reorg.assign(Strat='NA',IGV='NA',UNK='NA', Name=file, TruthSet="truth",SummaryType="summary",CompEng='VcfDist',IndelLen='NA')
vcfdist_final

,Type,MinQual,Precision,Recall,F1Score,QueryTP,TruthTP,QueryFP,TruthFN,Strat,IGV,UNK,Name,TruthSet,SummaryType,CompEng,IndelLen
0,SNP,0,0.992188,0.992873,0.992530,4514758,4514760,35458,32255,NA,NA,NA,hg001,truth,summary,VcfDist,NA
1,SNP,56,0.993706,0.992075,0.992890,4511135,4511137,28490,35878,NA,NA,NA,hg001,truth,summary,VcfDist,NA
2,INDEL,0,0.993758,0.991510,0.992633,663701,663711,3658,5110,NA,NA,NA,hg001,truth,summary,VcfDist,NA
3,INDEL,32,0.993891,0.991418,0.992653,663643,663653,3570,5168,NA,NA,NA,hg001,truth,summary,VcfDist,NA


In [26]:
vcfdist_final.set_index(["Type", "MinQual"])

Precision    Recall   F1Score  QueryTP  TruthTP  QueryFP  \
Type  MinQual                                                             
SNP   0         0.992188  0.992873  0.992530  4514758  4514760    35458   
      56        0.993706  0.992075  0.992890  4511135  4511137    28490   
INDEL 0         0.993758  0.991510  0.992633   663701   663711     3658   
      32        0.993891  0.991418  0.992653   663643   663653     3570   

               TruthFN Strat IGV UNK   Name TruthSet SummaryType  CompEng  \
Type  MinQual                                                               
SNP   0          32255    NA  NA  NA  hg001    truth     summary  VcfDist   
      56         35878    NA  NA  NA  hg001    truth     summary  VcfDist   
INDEL 0           5110    NA  NA  NA  hg001    truth     summary  VcfDist   
      32          5168    NA  NA  NA  hg001    truth     summary  VcfDist   

              IndelLen  
Type  MinQual           
SNP   0             NA  
      56            NA  
INDEL 0             NA  
      32            NA

In [27]:
vcfeval_goal[0:8]

,Precision,Recall,F1_Score,TP_Eval,TP_Base,FP,FN,Stratifier,IGV_Session,UNK,Name,Truth_Set,Summary_Type,Comparison_Engine,IndelLength
Type,,,,,,,,,,,,,,,
SNP,0.990900,0.994000,0.992500,3234532,3234812,29632,19540,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,772546.0,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
INDEL,0.988400,0.989200,0.988800,462828,462623,5452,5036,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,458723.0,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HetIndel,0.985112,0.987351,0.986230,284319,284204,4297,3641,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HomVarIndel,0.994636,0.993016,0.993825,177457,177437,957,1248,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HetSNP,0.985316,0.994106,0.989691,1954746,1954911,29131,11591,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
HomVarSNP,0.999576,0.993827,0.996693,1279800,1279844,543,7950,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,summary,VcfEval,NaN
NaN,0.989058,0.992816,0.990933,225244,225130,2492,1629,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,deletion,VcfEval,NaN
NaN,0.993076,0.991650,0.992362,209977,209956,1464,1768,NaN,gs://fc-7b82f4a6-46e8-4b46-928e-fd5a283db0a5/s...,NaN,illumina_novaseq6000_hg001_gatk,truth,insertion,VcfEval,NaN


In [28]:
vcfdist_final.to_csv(file+'/precision-recall-summary_'+file+'.csv')

In [29]:
print('Done!')

Done!
